# Funciones para la API

Se desarrollan y prueban las funciones que son solicitadas para la API. Luego, al momento del deployarlas es posible que sufran algunos cambios en función de los requerimientos propios de FastAPI o Render.

## Importaciones

In [6]:
import pandas as pd


import warnings
warnings.filterwarnings("ignore")

## Carga de los dataset necesarios

Se cargan los conjuntos de datos necesarios para llevar a cabo las consultas solicitadas. Se decide subir los datos de esta manera, y no los dataset completos, debido a la limitada capacidad de almacenamiento que permite Render. Por otra parte, considerando que el objetivo es entregar un PVM, a modo de prueba de concepto, se consideró adecuada esta simplificación de los datasets.

Los  dataset que se utilizan son:

* **df_reviews_unido**: contiene la información relacionada a los usuarios que realizan reviews de los juegos en formato de análisis de sentimientos. Entre esta información, se encuentran las recomendaciones o no del juego por parte de usuario, la fecha del review así como datos del usuario como su id, su url del perfil.
* **df_gastos_items**: contiene la cantidad de items que consume cada usuario y el precio de cada uno de los productos que consume.
* **df_playtime_forever**: contiene para cada usuario su identificación, la url de su perfil, la cantidad de horas jugadas por cada género de juego.
* **genre_ranking**: contiene el ranking de los géneros de juegos con mas horas jugadas.
* **df_items_developer**: contiene la información relacionada con cada item de juegos como su id, precio, desarrollador y año de lanzamiento.

In [9]:
# Usaremod pandas para leer los archivos Parquet
df_reviews = pd.read_parquet(r'C:\Users\melan\Desktop\DATA SCIENCE- Henry\PI1_MLOps_videojuegos\Data_cleaned\df_games_unido.parquet')
df_gastos_items = pd.read_parquet(r'C:\Users\melan\Desktop\DATA SCIENCE- Henry\PI1_MLOps_videojuegos\Data_cleaned\df_gastos_items_unido.parquet')
df_playtime_forever = pd.read_parquet(r'C:\Users\melan\Desktop\DATA SCIENCE- Henry\PI1_MLOps_videojuegos\Data_cleaned\df_playtime_forever_unido.parquet')
df_genre_ranking = pd.read_parquet(r'C:\Users\melan\Desktop\DATA SCIENCE- Henry\PI1_MLOps_videojuegos\Data_cleaned\df_genre_ranking_unido.parquet')
df_items_developer = pd.read_parquet(r'C:\Users\melan\Desktop\DATA SCIENCE- Henry\PI1_MLOps_videojuegos\Data_cleaned\df_items_developer_unido.parquet')

A continuación, se desarrollan cada una de las funciones solicitadas.

## userdata

Esta función tiene por parámentro 'user_id' y devulve la cantidad de dinero gastado por el usuario, el porcentaje de recomendaciones que realizó sobre la cantidad de reviews que se analizan y la cantidad de items que consume el mismo.

In [41]:
df_reviews

,genres,price,early_access,id,release_anio,publisher,app_name,title,developer
0,Action,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,Casual,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
2,Indie,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
3,Simulation,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
4,Strategy,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
...,...,...,...,...,...,...,...,...,...
71543,Indie,1.99,False,610660,2018,Laush Studio,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich
71544,Racing,1.99,False,610660,2018,Laush Studio,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich
71545,Simulation,1.99,False,610660,2018,Laush Studio,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich
71546,Casual,4.99,False,658870,2017,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,"xropi,stev3ns"


In [37]:
df_gastos_items


,items_count,user_id,price
0,58,--000--,397.78
1,44,--ace--,166.82
2,23,--ionex--,99.93
3,68,-2SV-vuLB-Kg,427.50
4,149,-404PageNotFound-,1469.34
...,...,...,...
70907,5,zzonci,19.98
70908,61,zzoptimuszz,64.98
70909,13,zzydrax,99.94
70910,84,zzyfo,828.51


In [32]:
def userdata(user_id):
    
    # Filtra por el usuario de interés
    usuario_reviews = df_reviews[df_reviews['user_id'] == user_id]
    usuario_gastos = df_gastos_items[df_gastos_items['user_id'] == user_id]
    
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = usuario_gastos['price'].sum()
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario_reviews['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews)
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    # Calcula el total de items que consume el usuario
    total_items = usuario_gastos['items_count'].sum()
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': total_items
    }


In [42]:
user_id = '761140'
resultados = userdata(user_id)
print(resultados)


KeyError: 'user_id'

In [17]:
def userdata(user_id):
    
    # Filtra por el usuario de interés
    usuario = df_reviews[df_reviews['id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews['id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items.astype(int)
    }

In [18]:
user_id = 'EchoXSilence'
userdata(user_id)

KeyError: 'id'

## countreviews

En esta función se ingresan dos fechas entre las que se quiere hacer una consulta y devuelve la cantidad de usuarios que realizaron reviews entre dichas fechas y el porcentaje de las recomendaciones positivas (True) que los mismos hicieron.

In [6]:
def countreviews(fecha_inicio, fecha_fin):
    
    # Filtra el dataframe entre las fechas de interés
    user_data_entre_fechas = df_reviews[(df_reviews['reviews_date'] >= fecha_inicio) & (df_reviews['reviews_date'] <= fecha_fin)]
    # Calcula la cantidad de usuarios que dieron reviews entre las fechas de interés
    total_usuarios = user_data_entre_fechas['user_id'].nunique()
    # Calcula el total de recomendaciones entre las fechas de interes (True + False)
    total_recomendacion = len(user_data_entre_fechas)
    # Calcula la cantidad de recomendaciones positivas que que hicieron entre las fechas de interés
    total_recomendaciones_True = user_data_entre_fechas['reviews_recommend'].sum()
    # Calcula el porcentaje de recomendación realizadas entre el total de usuarios
    porcentaje_recomendaciones = (total_recomendaciones_True / total_recomendacion) * 100
    
    return {
        'total_usuarios_reviews': total_usuarios,
        'porcentaje_recomendaciones': round(porcentaje_recomendaciones,2)
    }

In [7]:
countreviews('2011-11-05', '2012-12-24')

{'total_usuarios_reviews': 822, 'porcentaje_recomendaciones': 98.73}

## genre

Esta función recibe como parámetro un género de videojuego y devuelve el puesto en el que se encuentra dicho género sobre un ranking de los mismos analizando la cantidad de horas jugadas para cada uno.

In [8]:
def genre(genero):
    # Busca el ranking para el género de interés
    rank = genre_ranking[genre_ranking['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [9]:
genero = 'Simulation'
genre(genero)

{'rank': 5}

## userforgenre

Esta función recibe como parámetro el género de un videojuego y devuelve el top 5 de los usuarios con más horas de juego en el género ingresado, indicando el id del usuario y el url de su perfil.

In [10]:
def userforgenre(genero):
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'] == genero]
    # Agrupa el dataframe filtrado por usuario y suma la cantidad de horas
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    # Se hace un diccionario vacío para guardar los datos que se necesitan
    top_users_dict = {}
    for index, row in top_users.iterrows():
        # User info recorre cada fila del top 5 y lo guarda en el diccionario
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_dict[index + 1] = user_info
    
    return top_users_dict

In [11]:
genero = 'Action'
userforgenre(genero)

{1: {'user_id': 'Sp3ctre', 'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
 2: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 3: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 4: {'user_id': 'Terminally-Chill',
  'user_url': 'http://steamcommunity.com/id/Terminally-Chill'},
 5: {'user_id': 'DownSyndromeKid',
  'user_url': 'http://steamcommunity.com/id/DownSyndromeKid'}}

## developer

Esta función recibe como parámetro 'developer', que es la empresa desarrolladora del juego, y devuelve la cantidad de items que desarrolla dicha empresa y el porcentaje de contenido Free por año por sobre el total que desarrolla

In [12]:
def developer(desarrollador):
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('release_anio')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [13]:
desarrollador = 'Mechanical Boss'
developer(desarrollador)

{'cantidad_por_año': {'2016': 2, '2017': 1},
 'porcentaje_gratis_por_año': {'2016': 50, '2017': 0}}

## sentiment_analysis

Esta función recibe como parámetro el año de lanzamiento de un juego y según ese año devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento, como Negativo, Neutral y Positivo.

In [21]:
def sentiment_analysis(anio):
    # Filtrar las reseñas del desarrollador específico
    anio_reviews = df_reviews[df_reviews['release_anio'] == str(anio)]
    
    # Inicializar un diccionario para contar las categorías de sentimiento
    sentiment_counts = {'Negative': 0, 'Neutral': 0, 'Positive': 0}
    
    # Iterar a través de las reseñas del desarrollador
    for index, row in anio_reviews.iterrows():
        sentiment = row['sentiment_analysis']
        sentiment_category = ''
        
        # Asignar la categoría de sentimiento correspondiente
        if sentiment == 0:
            sentiment_category = 'Negative'
        elif sentiment == 1:
            sentiment_category = 'Neutral'
        elif sentiment == 2:
            sentiment_category = 'Positive'
        
        # Incrementar el contador correspondiente en el diccionario
        sentiment_counts[sentiment_category] += 1
    
    return sentiment_counts

In [22]:
anio = 2009
sentiment_analysis(anio)

{'Negative': 149, 'Neutral': 1178, 'Positive': 589}